# Treadmill experiments

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
from pathlib import Path

import scipy
from scipy.ndimage import zoom
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

from cottage_analysis.analysis import spheres
from cottage_analysis.plotting import depth_selectivity_plots

In [ ]:
project = "colasa_3d-vision_revisions"

In [ ]:
import flexiznam as flz

flm_sess = flz.get_flexilims_session(project_id=project)

mice = flz.get_entities(datatype="mouse", flexilims_session=flm_sess)
print(f"{len(mice)} mice in {project}")

In [ ]:
protocol_base = "SpheresTubeMotor"

treadmill_sessions = {}
all_sessions = flz.get_entities(datatype="session", flexilims_session=flm_sess)
for mouse_name, mouse_data in mice.iterrows():
    sessions = all_sessions[all_sessions.origin_id == mouse_data.id]
    for session_name, sess_data in sessions.iterrows():
        recordings = flz.get_children(
            parent_id=sess_data.id,
            flexilims_session=flm_sess,
            children_datatype="recording",
        )
        if not len(recordings):
            print(f"No recordings for session {session_name}")
            continue
        if protocol_base in recordings.protocol.values:
            treadmill_sessions[session_name] = recordings
print(f"{len(treadmill_sessions)} sessions with treadmill data")

In [ ]:
# Process control part normally
if False:
    for sess_name in treadmill_sessions.keys():
        from cottage_analysis.pipelines import pipeline_utils

        pipeline_filename = "run_analysis_pipeline.sh"
        conflicts = "overwrite"
        photodiode_protocol = 5
        pipeline_utils.sbatch_session(
            project=project,
            session_name=sess_name,
            pipeline_filename=pipeline_filename,
            conflicts=conflicts,
            photodiode_protocol=photodiode_protocol,
            run_depth_fit=True,
            run_rsof_fit=True,
            run_rf=False,
            protocol_base="SpheresPermTubeReward",
        )

In [ ]:
# Load neurons df
import flexiznam as flz

from cottage_analysis.pipelines import pipeline_utils

flexilims_session = flz.get_flexilims_session(project_id=project)
all_neurons_df = {}
for sess_name in treadmill_sessions.keys():
    try:
        neurons_ds = pipeline_utils.create_neurons_ds(
            session_name=sess_name,
            flexilims_session=flexilims_session,
            conflicts="skip",
        )
    except flz.FlexilimsError:
        print(f"Flexilims error for {sess_name}")
    try:
        neurons_df = pd.read_pickle(neurons_ds.path_full)
        all_neurons_df[sess_name] = neurons_df
        print(f"Loaded neurons df for {sess_name}")
    except FileNotFoundError as e:
        print(f"No neurons_df for {sess_name}, {e}")

In [ ]:
# Range of "normal" figure
log_range = {
    "rs_bin_log_min": 0,
    "rs_bin_log_max": 2.5,
    "rs_bin_num": 6,
    "of_bin_log_min": -1.5,
    "of_bin_log_max": 3.5,
    "of_bin_num": 11,
    "log_base": 10,
}
rs_bins = (
    np.logspace(
        log_range["rs_bin_log_min"],
        log_range["rs_bin_log_max"],
        num=log_range["rs_bin_num"],
        base=log_range["log_base"],
    )
    # / 100
)
rs_bins = np.insert(rs_bins, 0, 0)

of_bins = np.logspace(
    log_range["of_bin_log_min"],
    log_range["of_bin_log_max"],
    num=log_range["of_bin_num"],
    base=log_range["log_base"],
)
of_bins = np.insert(of_bins, 0, 0)


def rs2pos(x):
    return np.log10(x + 1) / np.log10(rs_bins.max() + 1) * (len(rs_bins)) - 1


def of2pos(x):
    rng = np.log10(of_bins.max() / of_bins[1])
    return np.log10(x / of_bins[1]) / (rng) * (len(of_bins) - 1)

In [ ]:

from cottage_analysis.analysis import treadmill


def load_tread_and_sphere(session_name, neurons_df):
    vs_df_tread, trials_df_tread = treadmill.sync_treadmill_sess(
        session_name, project, flexilims_session
    )

    # Compute response matrix
    motor_speeds, optic_flows, tread_responses = treadmill.compute_response_matrix(
        neurons_df, trials_df_tread
    )

    # load "normal" vs_df
    vs_df_example, trials_df_example = spheres.sync_all_recordings(
        session_name=session_name,
        flexilims_session=flexilims_session,
        project=project,
        filter_datasets={"anatomical_only": 3},
        recording_type="two_photon",
        protocol_base="SpheresPermTubeReward",
        photodiode_protocol=5,
        return_volumes=True,
    )
    suite2p_ds = flz.get_datasets_recursively(
        flexilims_session=flexilims_session,
        origin_name=session_name,
        dataset_type="suite2p_traces",
    )
    fs = list(suite2p_ds.values())[0][-1].extra_attributes["fs"]

    rs_arr = np.array([j for i in trials_df_example.RS_stim.values for j in i]) * 100
    of_arr = np.degrees([j for i in trials_df_example.OF_stim.values for j in i])
    return (
        motor_speeds,
        optic_flows,
        tread_responses,
        trials_df_example,
        fs,
        rs_arr,
        of_arr,
    )



In [ ]:


label_motor_speeds = 2 ** np.arange(2 - 1, 7)
label_optic_flows = np.hstack([1 / 4, 4 ** np.arange(6)])
fontsize_dict = {"title": 7, "label": 7, "tick": 5, "legend": 5}

In [ ]:
# PLOT ALL SINGLE NEURONS
if False:
    for session_name in all_neurons_df.keys():
        neurons_df = all_neurons_df[session_name]
        txt = f"{len(neurons_df)} neurons"
        good_neurons = neurons_df.query("is_depth_neuron == True")
        txt += f", {len(good_neurons)} depth selective"
        good_neurons = good_neurons.query(
            "depth_tuning_test_spearmanr_rval_closedloop > 0.4"
        )
        txt += f", {len(good_neurons)} spearman R >0.4"
        good_neurons = good_neurons.query(
            "depth_tuning_test_spearmanr_pval_closedloop < 0.05"
        )
        txt += f", {len(good_neurons)} spearman p <0.05"

        # add an estimate of peak of fit
        good_neurons["max_fit"] = good_neurons.depth_tuning_popt_closedloop_running.map(
            lambda x: np.exp(x[0])
        )
        good_neurons = good_neurons.query("max_fit < 10")
        good_neurons = good_neurons.query("max_fit > 0.1")
        txt += f", {len(good_neurons)} with large peak dff"
        print(txt)

        (
            motor_speeds,
            optic_flows,
            tread_responses,
            trials_df_example,
            fs,
            rs_arr,
            of_arr,
        ) = load_tread_and_sphere(session_name, neurons_df)
        two_seconds = np.floor(2 * fs).astype(int)
        # plot all good neurons
        save_folder = flz.get_processed_path(
            "colasa_3d-vision_revisions/analysis/treadmill_single_cells"
        )
        png_folder = save_folder / "small_png"
        png_folder.mkdir(exist_ok=True)

        warnings.filterwarnings("ignore", message="Mean of empty slice")
        # For NumPy 2.0+, you might also need/want to add:
        warnings.filterwarnings(
            "ignore", message="invalid value encountered in scalar divide"
        )
        fig = plt.figure(figsize=(5, 2.8))

        label_motor_speeds = 2 ** np.arange(2 - 1, 7)
        label_optic_flows = np.hstack([1 / 4, 4 ** np.arange(6)])
        cmap = "plasma"
        for row, roi_id in enumerate(np.arange(len(good_neurons))):

            fig.clear()
            neuron = good_neurons.iloc[roi_id]
            roi = neuron.name
            print(f"Plotting {roi}")
            motor_resp = tread_responses[roi]
            avg_last2 = motor_resp[..., -two_seconds:].mean(axis=-1)
            vmin = 0
            vmax = avg_last2.max()

            # plot normal tuning
            if True:
                ax = plt.subplot(2, 3, 1)
                depth_tuning_kwargs = dict(
                    rs_thr=None,
                    plot_fit=True,
                    plot_smooth=False,
                    linewidth=2,
                    linecolor="royalblue",
                    closed_loop=1,
                    fontsize_dict=fontsize_dict,
                    markersize=10,
                    markeredgecolor="w",
                )
                depth_selectivity_plots.plot_running_stationary_depth_tuning(
                    roi=roi,
                    roi_num=roi,
                    i=0,
                    ax=ax,
                    neurons_df=good_neurons,
                    trials_df=trials_df_example,
                    depth_tuning_kwargs=depth_tuning_kwargs,
                    fontsize_dict=fontsize_dict,
                    legend_loc="upper left",
                )
                ax0 = plt.subplot(1, 3, 2)

                dff_arr = np.vstack(trials_df_example.dff_stim.values)[:, roi]

                bin_means, rs_edges, of_edges, _ = scipy.stats.binned_statistic_2d(
                    x=rs_arr,
                    y=of_arr,
                    values=dff_arr,
                    statistic="mean",
                    bins=[rs_bins, of_bins],
                )
                vmax = max(np.nanmax(bin_means[1:, 1:]), vmax)
                im = ax0.imshow(
                    bin_means[1:, 1:].T,
                    origin="lower",
                    aspect="equal",
                    cmap=cmap,
                    vmin=vmin,
                    vmax=vmax,
                    extent=np.log10(
                        (rs_edges[1], rs_edges[-1], of_edges[1], of_edges[-1])
                    ),
                )
                toresize = bin_means[1:, 1:].T
                toresize[np.isnan(toresize)] = 0
                resized_matrix = zoom(toresize, 2, order=1)
                xs = np.linspace(
                    *np.log10([rs_edges[1], rs_edges[-1]]), resized_matrix.shape[1]
                )
                ys = np.linspace(
                    *np.log10([of_edges[1], of_edges[-1]]), resized_matrix.shape[0]
                )
                c = ax0.contour(
                    xs, ys, resized_matrix, cmap="Greys", levels=5, alpha=0.5
                )

            # plot motor
            ax = plt.subplot(1, 3, 3, sharex=ax0, sharey=ax0)

            im = ax.imshow(
                avg_last2.T,
                origin="lower",
                cmap=cmap,
                vmin=0,
                vmax=vmax,
                extent=np.log10(
                    [motor_speeds[0], motor_speeds[-1], optic_flows[0], optic_flows[-1]]
                ),
            )
            c = ax.contour(xs, ys, resized_matrix, cmap="Greys", levels=5, alpha=0.5)
            ax.set_xlim(-0.1, 2.1)
            ax.set_ylim(-1.6, 3.1)
            for x in [ax, ax0]:
                oflabels = [0.1, 1, 10, 100, 1000]
                x.set_xticks(
                    np.log10(motor_speeds),
                    labels=motor_speeds,
                    fontsize=fontsize_dict["tick"],
                )
                x.set_yticks(
                    np.log10(oflabels), labels=oflabels, fontsize=fontsize_dict["tick"]
                )
                x.set_xlabel("Motor speed (cm/s)", fontsize=fontsize_dict["label"])

            ax0.set_ylabel(
                "Optic flow speed (degrees/s)", fontsize=fontsize_dict["label"]
            )

            # Add an Axes to the right of the main Axes.
            cax = fig.add_axes([0.95, 0.5, 0.01, 0.3])
            l = np.round(np.linspace(0, vmax, 3), 1)
            fig.colorbar(im, cax=cax)
            cax.set_yticks(l, labels=l, fontsize=fontsize_dict["tick"])
            cax.set_ylabel(r"$\Delta$ F/F", fontsize=fontsize_dict["label"])
            cax.set_xticks([])

            plt.subplots_adjust(wspace=1)
            plt.suptitle(f"{session_name} roi {roi}")
            if True:
                fig.savefig(save_folder / f"{session_name}_roi{roi}.svg")
                fig.savefig(png_folder / f"{session_name}_roi{roi}.png", dpi=75)

In [ ]:
# Load an example session
session_name = 'PZAG17.3a_S20250402'
neurons_df = all_neurons_df[session_name]
txt = f"{len(neurons_df)} neurons"
good_neurons = neurons_df.query("is_depth_neuron == True")
txt += f", {len(good_neurons)} depth selective"
good_neurons = good_neurons.query(
    "depth_tuning_test_spearmanr_rval_closedloop > 0.4"
)
txt += f", {len(good_neurons)} spearman R >0.4"
good_neurons = good_neurons.query(
    "depth_tuning_test_spearmanr_pval_closedloop < 0.05"
)
txt += f", {len(good_neurons)} spearman p <0.05"

# add an estimate of peak of fit
good_neurons["max_fit"] = good_neurons.depth_tuning_popt_closedloop_running.map(
    lambda x: np.exp(x[0])
)
good_neurons = good_neurons.query("max_fit < 10")
good_neurons = good_neurons.query("max_fit > 0.1")
txt += f", {len(good_neurons)} with large peak dff"
print(txt)

(
    motor_speeds,
    optic_flows,
    tread_responses,
    trials_df_example,
    fs,
    rs_arr,
    of_arr,
) = load_tread_and_sphere(session_name, neurons_df)
two_seconds = np.floor(2 * fs).astype(int)

In [ ]:
roi = 163


motor_resp = tread_responses[roi]
avg_last2 = motor_resp[..., -two_seconds:].mean(axis=-1)
vmin = 0
vmax = avg_last2.max()

fig = plt.figure()
# plot normal tuning
if False:
    ax = plt.subplot(2, 3, 1)
    depth_tuning_kwargs = dict(
        rs_thr=None,
        plot_fit=True,
        plot_smooth=False,
        linewidth=2,
        linecolor="royalblue",
        closed_loop=1,
        fontsize_dict=fontsize_dict,
        markersize=10,
        markeredgecolor="w",
    )
    depth_selectivity_plots.plot_running_stationary_depth_tuning(
        roi=roi,
        roi_num=roi,
        i=0,
        ax=ax,
        neurons_df=good_neurons,
        trials_df=trials_df_example,
        depth_tuning_kwargs=depth_tuning_kwargs,
        fontsize_dict=fontsize_dict,
        legend_loc="upper left",
    )
ax0 = plt.subplot(1, 3, 2)

dff_arr = np.vstack(trials_df_example.dff_stim.values)[:, roi]

bin_means, rs_edges, of_edges, _ = scipy.stats.binned_statistic_2d(
    x=rs_arr,
    y=of_arr,
    values=dff_arr,
    statistic="mean",
    bins=[rs_bins, of_bins],
)
vmax = max(np.nanmax(bin_means[1:, 1:]), vmax)
im = ax0.imshow(
    bin_means[1:, 1:].T,
    origin="lower",
    aspect="equal",
    cmap=cmap,
    vmin=vmin,
    vmax=vmax,
    extent=np.log10(
        (rs_edges[1], rs_edges[-1], of_edges[1], of_edges[-1])
    ),
)
toresize = bin_means[1:, 1:].T
toresize[np.isnan(toresize)] = 0
resized_matrix = zoom(toresize, 2, order=1)
xs = np.linspace(
    *np.log10([rs_edges[1], rs_edges[-1]]), resized_matrix.shape[1]
)
ys = np.linspace(
    *np.log10([of_edges[1], of_edges[-1]]), resized_matrix.shape[0]
)
if False:
    c = ax0.contour(
        xs, ys, resized_matrix, cmap="Greys", levels=5, alpha=0.5
    )

# plot motor
ax = plt.subplot(1, 3, 3, sharex=ax0, sharey=ax0)

im = ax.imshow(
    avg_last2.T,
    origin="lower",
    cmap=cmap,
    vmin=0,
    vmax=vmax,
    extent=np.log10(
        [motor_speeds[0], motor_speeds[-1], optic_flows[0], optic_flows[-1]]
    ),
)
if False:
    c = ax.contour(xs, ys, resized_matrix, cmap="Greys", levels=5, alpha=0.5)
ax.set_xlim(-0.1, 2.1)
ax.set_ylim(-1.6, 3.1)
for x in [ax, ax0]:
    oflabels = [0.1, 1, 10, 100, 1000]
    x.set_xticks(
        np.log10(motor_speeds),
        labels=motor_speeds,
        fontsize=fontsize_dict["tick"],
    )
    x.set_yticks(
        np.log10(oflabels), labels=oflabels, fontsize=fontsize_dict["tick"]
    )
    x.set_xlabel("Motor speed (cm/s)", fontsize=fontsize_dict["label"])

ax0.set_ylabel(
    "Optic flow speed (degrees/s)", fontsize=fontsize_dict["label"]
)

# Add an Axes to the right of the main Axes.
cax = fig.add_axes([0.95, 0.3, 0.01, 0.3])
l = np.round(np.linspace(0, vmax, 3), 1)
fig.colorbar(im, cax=cax)
cax.set_yticks(l, labels=l, fontsize=fontsize_dict["tick"])
cax.set_ylabel(r"$\Delta$ F/F", fontsize=fontsize_dict["label"])
cax.set_xticks([])

plt.subplots_adjust(wspace=0.5)


In [ ]:
ax0 = plt.subplot(1,3,1)
original = np.array(bin_means[1:, 1:].T)
new = np.zeros_like(original)
im = ax0.imshow(
    new,
    origin="lower",
    aspect="equal",
    cmap=cmap,
    vmin=vmin,
    vmax=vmax,
    extent=np.log10(
        (rs_edges[1], rs_edges[-1], of_edges[1], of_edges[-1])
    ),
)
ax0.set_xlim(-0.1, 2.1)
ax0.set_ylim(-1.6, 3.1)
oflabels = [0.1, 1, 10, 100, 1000]
ax0.set_xticks(
    np.log10(motor_speeds),
    labels=motor_speeds,
    fontsize=fontsize_dict["tick"],
)
ax0.set_yticks(
    np.log10(oflabels), labels=oflabels, fontsize=fontsize_dict["tick"]
)
ax0.set_xlabel("Motor speed (cm/s)", fontsize=fontsize_dict["label"])

ax0.set_ylabel(
    "Optic flow speed (degrees/s)", fontsize=fontsize_dict["label"]
)


In [ ]:
protocol_base = "SpheresPermTubeReward"
SESSION = session_name
for session_name, recordings in treadmill_sessions.items():
    # Get the recordings that are  motor
    if session_name != SESSION:
        continue
    recordings = recordings[recordings.protocol != protocol_base]
    print(f"{len(recordings)} sphere protocols")
    print(recordings.iloc[0].name)
    break

In [ ]:
import flexiznam as flz
from cottage_analysis.analysis.spheres import *
import flexiznam as flz


exclude_datasets = None

harp_is_in_recording = True
use_onix = False
conflicts = "skip"
sync_kwargs = None
ephys_kwargs = None
# We can just run the same pipeline. It will skip depth and rsof fit and just run the
# the rf fit
protocol_base = "SpheresPermTubeReward"
flexilims_session = flz.get_flexilims_session(project_id=project)
assert flexilims_session is not None or project is not None

In [ ]:
from cottage_analysis.analysis import treadmill

filter_datasets = {"anatomical_only": 3}
recording_type = "two_photon"
photodiode_protocol = 5
return_volumes = True


load_onix = False if recording_type == "two_photon" else True
all_imaging_df = []
for i, recording_name in enumerate(sorted(recordings.name)):
    print(f"Processing recording {i+1}/{len(recordings)}")
    recording, harp_recording, onix_rec = get_relevant_recordings(
        recording_name, flexilims_session, harp_is_in_recording, load_onix
    )
    vs_df = synchronisation.generate_vs_df(
        recording=recording,
        photodiode_protocol=photodiode_protocol,
        flexilims_session=flexilims_session,
        harp_recording=harp_recording,
        onix_recording=onix_rec if use_onix else None,
        project=project,
        conflicts=conflicts,
        sync_kwargs=sync_kwargs,
        protocol_base=protocol_base,
    )
    imaging_df = synchronisation.generate_imaging_df(
        vs_df=vs_df,
        recording=recording,
        flexilims_session=flexilims_session,
        filter_datasets=filter_datasets,
        exclude_datasets=exclude_datasets,
        return_volumes=return_volumes,
    )
    imaging_df = format_imaging_df(imaging_df=imaging_df, recording=recording)
    imaging_df = treadmill.process_imaging_df(imaging_df, trial_duration=2)
    all_imaging_df.append(imaging_df)

In [ ]:
imaging_df = pd.concat(all_imaging_df, ignore_index=True)

In [ ]:
import matplotlib.pyplot as plt

starts = imaging_df.query("is_trial_start")
ends = imaging_df.query("is_trial_end")

plt.figure(figsize=(7, 5))
t0 = imaging_df.imaging_harptime.min()
for i in range(2):
    plt.subplot(2, 1, 1 + i)
    plt.plot(imaging_df.imaging_harptime - t0, imaging_df.MotorSpeed, label="Motor")
    plt.plot(imaging_df.imaging_harptime - t0, imaging_df.RS * 100, label="Actual")
    plt.ylabel("Speed (cm/s)")
    plt.ylim(-5, 70)
    plt.scatter(ends.imaging_harptime - t0, ends.MotorSpeed, color="k")
    plt.scatter(starts.imaging_harptime - t0, starts.MotorSpeed, color="g")
plt.xlim(100, 255)
plt.xlabel("Time (s)")
plt.axvline(163, color="k", zorder=-10)
plt.axvline(165, color="k", zorder=-10)
plt.legend(loc="upper right")
plt.tight_layout()

In [ ]:
from cottage_analysis.analysis import spheres
from cottage_analysis.analysis import treadmill

vs_df, trials_df = treadmill.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    photodiode_protocol=5,
    return_volumes=True,
)

In [ ]:
trials_df.shape

In [ ]:
trials_df["MotorSpeed"] = np.round(
    treadmill.sps2speed(trials_df["MotorSps_stim"].apply(np.nanmedian).values)
)
trials_df["expected_optic_flow"] = np.round(
    trials_df["expected_optic_flow_stim"].apply(np.nanmedian).values
)

trials_df.groupby(["MotorSpeed", "expected_optic_flow"]).aggregate(len)[
    "trial_no"
].unstack()

In [ ]:
suite2p_ds = flz.get_datasets_recursively(
    flexilims_session=flexilims_session,
    origin_name=session_name,
    dataset_type="suite2p_traces",
)
suite2p_ds = list(suite2p_ds.values())[0][-1]
fs = suite2p_ds.extra_attributes["fs"]

In [ ]:
suite2p_ds.path_full

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm

fig_folder = suite2p_ds.path_full.parent.parent / "motor_analysis"
fig_folder.mkdir(exist_ok=True)

motor_speeds = 2 ** np.arange(2, 7)
optic_flows = 4 ** np.arange(6)
all_dffs = np.vstack(trials_df.dff_stim.values)
total_mean = np.nanmean(all_dffs, axis=0)
total_std = np.nanstd(all_dffs, axis=0)

roi = 514
plot_all_rois = False
if plot_all_rois:
    fig, axes = plt.subplots(len(optic_flows), len(motor_speeds), figsize=(10, 10))
    nrois = trials_df.dff_stim.iloc[0].shape[1]
    for roi in tqdm(range(nrois)):
        ymin = 0
        ymax = 0.05

        for x in axes.flatten():
            x.clear()
        for (motor, optic_flow), df in trials_df.groupby(
            ["MotorSpeed", "expected_optic_flow"]
        ):
            dffs = df.dff_stim.values
            shapes = np.vstack([dff.shape for dff in dffs])
            m = np.min(shapes[:, 0])
            dffs = np.stack([dff[:m, :] for dff in dffs])
            avg = np.nanmean(dffs, axis=0)
            std = np.nanstd(dffs, axis=0)

            m_index = list(motor_speeds).index(motor)
            of_index = list(optic_flows).index(optic_flow)
            ax = axes[len(optic_flows) - of_index - 1, m_index]
            # ax.set_title(f"M: {motor}, OF: {optic_flow}")
            time = np.arange(avg.shape[0]) / fs
            # ax.fill_between(time, avg[:,roi]-std[:,roi], avg[:,roi]+std[:,roi], alpha=0.5, color='k')
            ax.plot(time, dffs[..., roi].T, color="k", lw=1, alpha=0.7)
            ax.plot(time, avg[:, roi], color="darkorchid", lw=2)
            ax.set_xlim(time.min(), time.max())
            ymin = min(ymin, dffs[..., roi].min())
            ymax = max(ymax, dffs[..., roi].max())

        for x in axes.flatten():
            x.set_ylim(ymin, ymax * 1.05)
        for i, m in enumerate(motor_speeds):
            axes[0, i].set_title(f"{int(m)} cm/s")
            axes[-1, i].set_xlabel("Time (s)")
        for i, m in enumerate(optic_flows):
            axes[len(optic_flows) - i - 1, 0].set_ylabel(f"OF: {int(m)} deg/s \n dF/F")
        fig.suptitle(f"{session_name} roi {roi}")
        fig.tight_layout()
        fig.savefig(fig_folder / f"psth_roi{roi}.png")

In [ ]:
stim_df = imaging_df.query("is_stim")
n_frames = stim_df.groupby(["MotorSpeed", "expected_optic_flow"]).dffs.aggregate(len)
avg = stim_df.groupby(["MotorSpeed", "expected_optic_flow"]).aggregate("mean")

motor_speeds = 2 ** np.arange(2, 7)
optic_flows = 4 ** np.arange(6)


def compute_motor_of_matrix(df, motor_speeds=motor_speeds, optic_flows=optic_flows):
    avg = df.groupby(["MotorSpeed", "expected_optic_flow"]).aggregate("mean")
    data = avg.dffs.unstack()
    n_neurons = df.dffs.iloc[0].shape[1]
    output = np.zeros([len(motor_speeds), len(optic_flows), n_neurons])
    for ispeed, speed in enumerate(motor_speeds):
        for iof, optic_flow in enumerate(optic_flows):
            output[ispeed, iof, :] = data.loc[speed, optic_flow][0, :]
    return output


output = compute_motor_of_matrix(stim_df)

In [ ]:
# Compute global mean and std for z-scoring
all_dffs = np.dstack(imaging_df.dffs)[0]
total_mean = np.nanmean(all_dffs, axis=1)
total_std = np.nanstd(all_dffs, axis=1)
zscores_max = (np.max(output, axis=(0, 1)) - total_mean) / total_std
order = np.argsort(zscores_max)[::-1]

In [ ]:
roi = 514

for (motor, of), df in stim_df.groupby(["MotorSpeed", "expected_optic_flow"]):
    dff = np.dstack(df.dffs.values)
dff.shape

In [ ]:
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

min_motor = 0
range_motor = np.diff(np.log2(motor_speeds)[[0, -1]])[0]
range_of = np.diff(np.log2(optic_flows)[[0, -1]])[0]
min_of = 0
print(f"Motor range: {range_motor}, optic flow range: {range_of}")
fig, ax = plt.subplots(1, 1)

ax_cb = make_axes_locatable(ax)
# Add an Axes to the right of the main Axes.
cax1 = ax_cb.append_axes("right", size="7%", pad="2%")

data = output[..., roi]
img = ax.imshow(
    data.T,
    origin="lower",
    vmin=0,
    vmax=data.max(),
    cmap="viridis",
    extent=(min_motor, range_motor, min_of, range_of),
)
ax.set_xticks(np.linspace(min_motor + 0.5, range_motor - 0.5, len(motor_speeds)))
ax.set_xticklabels(motor_speeds)
ax.set_yticks(np.linspace(min_of + 0.5, range_of - 0.5, len(optic_flows)))
ax.set_yticklabels(optic_flows)
cb1 = fig.colorbar(img, cax=cax1)
cb1.set_label("DFF")
ax.set_xlabel("Motor speed (cm/s)")
ax.set_ylabel("Optic flow (degrees/s)")

In [ ]:
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable


min_motor = 0
range_motor = np.diff(np.log2(motor_speeds)[[0, -1]])[0]
range_of = np.diff(np.log2(optic_flows)[[0, -1]])[0]
min_of = 0
print(f"Motor range: {range_motor}, optic flow range: {range_of}")
fig, ax = plt.subplots(1, 1)
ax_cb = make_axes_locatable(ax)
# Add an Axes to the right of the main Axes.
cax1 = ax_cb.append_axes("right", size="7%", pad="2%")


data = (output[..., roi] - total_mean[roi]) / total_std[roi]
img = ax.imshow(
    data.T,
    origin="lower",
    vmin=-data.max(),
    vmax=data.max(),
    cmap="RdBu_r",
    extent=(min_motor, range_motor, min_of, range_of),
)
ax.set_xticks(np.linspace(min_motor + 0.5, range_motor - 0.5, len(motor_speeds)))
ax.set_xticklabels(motor_speeds)
ax.set_yticks(np.linspace(min_of + 0.5, range_of - 0.5, len(optic_flows)))
ax.set_yticklabels(optic_flows)
cb1 = fig.colorbar(img, cax=cax1)
cb1.set_label("Zscored DFF")
ax.set_xlabel("Motor speed (cm/s)")
ax.set_ylabel("Optic flow (degrees/s)")

In [ ]:
aspect_ratio = range_motor / range_of
fig, axes = plt.subplots(25, 25)
fig.set_size_inches(20, 20 / aspect_ratio)
do_zscore = True
for iax, ax in enumerate(axes.flatten()):
    roi = order[iax]
    if do_zscore:
        data = (output[..., roi] - total_mean[roi]) / total_std[roi]
        m = max(2, data.max())
        vmin = -m
        vmax = m
        cmap = "RdBu_r"
    else:
        data = output[..., roi]
        vmax = max(output[..., roi].max(), 0.1)
        vmin = 0
        cmap = "viridis"
    im = ax.imshow(
        data.T,
        origin="lower",
        vmax=vmax,
        vmin=vmin,
        cmap=cmap,
        extent=(min_motor, range_motor, min_of, range_of),
    )
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(roi)
plt.subplots_adjust(wspace=0, hspace=0.01)
plt.tight_layout()

In [ ]:
s = vis_stim_ds.path_full / vis_stim_ds.extra_attributes["csv_files"]["FrameLog"]
frame_df = pd.read_csv(s)
frame_df.head()